<a href="https://colab.research.google.com/github/ankesh86/IntroductionToMachineLearning/blob/main/DataPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install transformers==4.31.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 36.2 MB/s eta 0:00:00


In [ ]:
!pip -q install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 1.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.


In [ ]:
!parquet-tools inspect train_data.parquet


/bin/bash: line 1: parquet-tools: command not found


In [ ]:
from datasets import load_dataset

ds = load_dataset("Hello-SimpleAI/HC3", "all")

In [ ]:
ds.shape

{'train': (24322, 5)}

In [ ]:
train_dataset = ds['train']
train_df = train_dataset.to_pandas()
print(train_df.head())

  id                                           question  \
0  0  Why is every book I hear about a " NY Times # ...   
1  1  If salt is so bad for cars , why do we use it ...   
2  2  Why do we still have SD TV channels when HD lo...   
3  3  Why has nobody assassinated Kim Jong - un He i...   
4  4  How was airplane technology able to advance so...   

                                       human_answers  \
0  [Basically there are many categories of " Best...   
1  [salt is good for not dying in car crashes and...   
2  [The way it works is that old TV stations got ...   
3  [You ca n't just go around assassinating the l...   
4  [Wanting to kill the shit out of Germans drive...   

                                     chatgpt_answers       source  
0  [There are many different best seller lists th...  reddit_eli5  
1  [Salt is used on roads to help melt ice and sn...  reddit_eli5  
2  [There are a few reasons why we still have SD ...  reddit_eli5  
3  [It is generally not acceptable o

In [ ]:
human_df = train_df[['id', 'human_answers']].rename(columns={'id': 'prompt_id', 'human_answers': 'gen_text'})

chatgpt_df = train_df[['id', 'chatgpt_answers']].rename(columns={'id': 'prompt_id', 'chatgpt_answers': 'gen_text'})

In [ ]:
import pandas as pd

human_df['label'] = pd.Series(0, index=human_df.index)
chatgpt_df['label'] = pd.Series(1, index=chatgpt_df.index)

In [ ]:
final_df = pd.concat([human_df, chatgpt_df], axis=0)
final_df = final_df.reset_index(drop=True)  # Reset index and drop the old one
print(final_df)

      prompt_id                                           gen_text  label
0             0  [Basically there are many categories of " Best...      0
1             1  [salt is good for not dying in car crashes and...      0
2             2  [The way it works is that old TV stations got ...      0
3             3  [You ca n't just go around assassinating the l...      0
4             4  [Wanting to kill the shit out of Germans drive...      0
...         ...                                                ...    ...
48639     24317  [It's not uncommon for blood pressure to fluct...      1
48640     24318  [There are several possible causes of a painle...      1
48641     24319  [It is not appropriate for me to recommend a s...      1
48642     24320  [It is not uncommon for people with rheumatoid...      1
48643     24321  [It is not uncommon to experience back pain, e...      1

[48644 rows x 3 columns]


In [ ]:
counts = final_df.groupby('label').size()
print("Counts:")
print(counts)

Counts:
label
0    24322
1    24322
dtype: int64


In [ ]:
final_df.head(10)

,prompt_id,gen_text,label
0,0,"[Basically there are many categories of "" Best...",0
1,1,[salt is good for not dying in car crashes and...,0
2,2,[The way it works is that old TV stations got ...,0
3,3,[You ca n't just go around assassinating the l...,0
4,4,[Wanting to kill the shit out of Germans drive...,0
5,5,[Melanin ! Many of the the first known humans ...,0
6,6,[Because you 're a minor and your parents get ...,0
7,7,[It 's three fold : * Stuff is cheaper to mass...,0
8,8,"[EDIT , Nov 21 : By popular demand , now inclu...",0
9,9,"[Like viruses , they are ( groups of ) molecul...",0


In [ ]:
import pandas as pd

def join_all_items(text_list):
    return ' '.join(text_list)  # Join all items into a single string

final_df['text'] = final_df['gen_text'].apply(join_all_items)

print(final_df[['gen_text', 'text']])

                                                gen_text  \
0      [Basically there are many categories of " Best...   
1      [salt is good for not dying in car crashes and...   
2      [The way it works is that old TV stations got ...   
3      [You ca n't just go around assassinating the l...   
4      [Wanting to kill the shit out of Germans drive...   
...                                                  ...   
48639  [It's not uncommon for blood pressure to fluct...   
48640  [There are several possible causes of a painle...   
48641  [It is not appropriate for me to recommend a s...   
48642  [It is not uncommon for people with rheumatoid...   
48643  [It is not uncommon to experience back pain, e...   

                                                    text  
0      Basically there are many categories of " Best ...  
1      salt is good for not dying in car crashes and ...  
2      The way it works is that old TV stations got a...  
3      You ca n't just go around assassinat

In [ ]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48644 entries, 0 to 48643
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   prompt_id  48644 non-null  object
 1   gen_text   48644 non-null  object
 2   label      48644 non-null  int64 
 3   text       48644 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.5+ MB


# **Data pre-processing**

## **Data Lowercase**

In [ ]:
def to_lowercase(text):
    if isinstance(text, str):
        return text.lower()
    elif isinstance(text, (list, tuple)):
        return ' '.join([to_lowercase(elem) for elem in text])
    else:
        return text

final_df['text'] = final_df['text'].apply(to_lowercase)

In [ ]:
import re

def remove_punctuation(text):
    # Replace all non-alphanumeric characters with an empty string
    return re.sub(r'[^\w\s]', '', text) if type(text) == str else text

final_df['text'] = final_df['text'].apply(remove_punctuation)

In [ ]:
df = final_df[['text','label','prompt_id']]
df.to_csv('outputDataset.csv', index=False)

In [ ]:
final_df.isnull().sum()

prompt_id    0
gen_text     0
label        0
text         0
dtype: int64

# **Tokenization**

In [ ]:
!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2024-06-18 20:38:41--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-06-18 20:38:41--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.02MB/s    in 2m 39s  

2024-06-18 20:41:21 (5.16 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflatin

## Loading a Pre-trained GloVe Model Using Pandas

In the provided code snippet, you're loading a pre-trained GloVe (Global Vectors for Word Representation) model from a text file and converting it into a dictionary for easier access. Below is a breakdown of the parameters used in the `read_csv` function from pandas:

### Parameters Explained

- **`sep=" "`**: Specifies the delimiter that separates each field in the dataset. For GloVe files, each line starts with a word followed by its embedding, with each field separated by space.

- **`quoting=3`**: This parameter controls how quote characters are handled during parsing. Here, `3` corresponds to `csv.QUOTE_NONE`, meaning that quote characters in the input data do not signify the start or end of a quoted item. This prevents potential issues as some words might contain quote characters.

- **`header=None`**: Indicates to pandas that the first line in the file does not contain header information. For GloVe files, the first line includes actual data (a word and its corresponding vector), making it crucial to set this parameter to `None`.

- **`index_col=0`**: Sets the first column of the data, which contains the words, as the index of the DataFrame. This configuration facilitates easy access to the vectors by word when transforming the DataFrame into a dictionary.

### Dictionary Conversion

- **`glove_model = {key:value.values for key, value in glove_df.T.items()}`**: This line of code transforms the DataFrame into a dictionary for faster lookup. The DataFrame is transposed (`glove_df.T`) to switch rows and columns. In the transposed DataFrame, each word (now a column) and its vector form key-value pairs. The `.items()` method iterates over these pairs, with `key` being the word and `value.values` extracting the numerical array (embedding vector) associated with the word.

### Summary

This setup is typical for handling word embeddings in natural language processing tasks, where rapid access to the vector representation of words is often required. The resulting dictionary provides an efficient way to manage and utilize the GloVe embeddings for computational tasks in NLP.


In [ ]:
##dropping duplicates and keeping first records
unique_df = final_df.drop_duplicates(subset=['text'], keep='first')

In [ ]:
unique_df.shape

(45749, 4)

In [ ]:
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    # Ensure text is a string
    if isinstance(text, str):
        # Tokenize the text into words
        words = text.split()
        # Remove stop words
        filtered_words = [word for word in words if word.lower() not in stop_words]
        # Join words back into a single string
        return " ".join(filtered_words)
    return text

unique_df.loc[:, 'text'] = unique_df['text'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import numpy as np

def load_glove_model(glove_file):
    print("Loading Glove Model")
    with open(glove_file, 'r', encoding='utf8') as f:
        glove_model = {}
        for line in f:
            split_line = line.split()
            word = split_line[0]
            embedding = np.array([float(val) for val in split_line[1:]])
            glove_model[word] = embedding
        print("Done.", len(glove_model), " words loaded!")
    return glove_model

glove_path = 'glove.6B.300d.txt'
glove_model = load_glove_model(glove_path)

Loading Glove Model
Done. 400000  words loaded!


In [ ]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')  # Download the tokenizer model

def process_text(text):
    try:
        # Tokenize text and convert to lower case
        words = word_tokenize(text.lower())
        return words
    except TypeError:
        print(f"Warning: Non-string input encountered: {text}")
        return []

# Apply the processing function using .loc for safer operation
#unique_df.loc[:, 'tokenized_text'] = unique_df['text'].apply(process_text)
# Apply the processing function using .loc for safer operation
unique_df.loc[:, 'tokenized_text'] = unique_df['text'].apply(process_text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
unique_df['tokenized_text']

0        [basically, many, categories, best, seller, re...
1        [salt, good, dying, car, crashes, car, crashes...
2        [way, works, old, tv, stations, got, certain, ...
3        [ca, nt, go, around, assassinating, leaders, c...
4        [wanting, kill, shit, germans, drives, innovat...
                               ...                        
48639    [uncommon, blood, pressure, fluctuate, through...
48640    [several, possible, causes, painless, lump, ri...
48641    [appropriate, recommend, specific, medication,...
48642    [uncommon, people, rheumatoid, arthritis, ra, ...
48643    [uncommon, experience, back, pain, especially,...
Name: tokenized_text, Length: 45749, dtype: object

In [ ]:
import numpy as np

def tokens_to_vector(tokens, glove_model):
    vector_list = [glove_model[token] for token in tokens if token in glove_model]
    if vector_list:
        # Average the vectors to create a single vector per text
        text_vector = np.mean(vector_list, axis=0)
    else:
        # If no tokens in the text are in the glove_model, return a zero vector
        text_vector = np.zeros(300)  # Adjust the size depending on the GloVe vectors you loaded
    return text_vector

unique_df.loc[:, 'vector_features'] = unique_df['tokenized_text'].apply(lambda x: tokens_to_vector(x, glove_model))

In [ ]:
unique_df['vector_features'].head()

0    [-0.1312311344537815, 0.1357034411764706, 0.04...
1    [-0.062339281865285014, 0.0825920663212435, -0...
2    [-0.2674947338403042, 0.1201015755665399, 0.06...
3    [-0.14290759238095238, 0.0557432177142857, -0....
4    [-0.008924468421052642, 0.060109743609022576, ...
Name: vector_features, dtype: object

In [103]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

def calculate_perplexity(text, tokenizer, model, stride=512, max_length=1024):
    encodings = tokenizer(text, return_tensors='pt')

    nlls = []
    for i in range(0, encodings.input_ids.size(1), stride):
        begin_loc = max(i + stride - max_length, 0)
        end_loc = min(i + stride, encodings.input_ids.size(1))
        trg_len = end_loc - i  # may be different from stride on last loop
        input_ids = encodings.input_ids[:, begin_loc:end_loc]
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            log_likelihood = outputs[0] * trg_len

        nlls.append(log_likelihood)

    ppl = torch.exp(torch.stack(nlls).sum() / encodings.input_ids.size(1))
    return ppl.item()


# Example text that might be longer than the max length
long_text = unique_df['text'].iloc[4000]
print("Perplexity:", calculate_perplexity(long_text, tokenizer, model))
unique_df.loc[:, 'Perplexity'] = unique_df['text'].apply(lambda x: calculate_perplexity(x, tokenizer, model))

Perplexity: 1036.25732421875


Token indices sequence length is longer than the specified maximum sequence length for this model (1107 > 1024). Running this sequence through the model will result in indexing errors


KeyboardInterrupt: 

In [ ]:
def calculate_burstiness(text_data):
    lengths = [len(text.split()) for text in text_data]
    mean_length = np.mean(lengths)
    std_dev_length = np.std(lengths)
    burstiness = std_dev_length / mean_length if mean_length > 0 else 0
    return burstiness

# Example usage
texts = ["The quick brown fox", "jumps over the lazy dog", "in a quaint little cafe"]
print("Burstiness:", calculate_burstiness(texts))
unique_df.loc[:, 'Burstiness'] = unique_df['text'].apply(calculate_burstiness)

Burstiness: 0.10101525445522107


In [ ]:
from sklearn.model_selection import train_test_split

y=final_df['label']

X_train, X_test, y_train, y_test = train_test_split(final_df['text'], y, test_size=0.2, random_state=42)